# Create Academy of Finland Awards from Research.fi API

Creates Academy of Finland (Research Council of Finland) awards from the Research.fi API. ~5.4K projects.

**Prerequisites:**
- Run `scripts/local/academy_of_finland_to_s3.py` to download and upload the data first.

**Data source:** https://research.fi/en/results/fundings?funder=0245893-9  
**API:** https://researchfi-api-production.2.rahtiapp.fi/portalapi/funding/_search  
**S3 location:** `s3a://openalex-ingest/awards/academy_of_finland/academy_of_finland_projects.parquet`

**Academy of Finland funder:**
- funder_id: 4320321108
- ROR: https://ror.org/05k73zm37
- DOI: 10.13039/501100002341
- display_name: "Academy of Finland" (now officially "Research Council of Finland")

## Step 1: Create Staging Table from S3

In [ ]:
%sql
-- Create the staging table from S3 parquet
CREATE OR REPLACE TABLE openalex.awards.academy_of_finland_raw
USING delta
AS
SELECT
    *,
    current_timestamp() as databricks_ingested_at
FROM parquet.`s3a://openalex-ingest/awards/academy_of_finland/academy_of_finland_projects.parquet`;

In [ ]:
%sql
-- Check row count (should be ~5.4K)
SELECT COUNT(*) as total_projects FROM openalex.awards.academy_of_finland_raw;

In [ ]:
%sql
-- Inspect column names and sample data
DESCRIBE openalex.awards.academy_of_finland_raw;

In [ ]:
%sql
-- Sample the raw data
SELECT * FROM openalex.awards.academy_of_finland_raw LIMIT 5;

## Step 2: Create Academy of Finland Awards Table

In [ ]:
%sql
CREATE OR REPLACE TABLE openalex.awards.academy_of_finland_awards
USING delta
AS
WITH
-- Get Academy of Finland funder from OpenAlex
aka_funder AS (
    SELECT
        funder_id,
        display_name,
        ror_id,
        doi
    FROM openalex.common.funder
    WHERE funder_id = 4320321108  -- Academy of Finland
),

awards_transformed AS (
    SELECT
        -- Generate unique ID using xxhash64 of funder_id:project_id
        abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(CAST(g.project_id AS STRING))))) % 9000000000 as id,

        -- Display name = project title (prefer English, fallback to Finnish)
        COALESCE(NULLIF(TRIM(g.title_en), ''), g.title_fi) as display_name,

        -- Description (prefer English, fallback to Finnish)
        COALESCE(NULLIF(TRIM(g.description_en), ''), g.description_fi) as description,

        -- Funder info
        f.funder_id,
        g.funder_project_number as funder_award_id,

        -- Amount in EUR
        TRY_CAST(g.amount_eur AS DOUBLE) as amount,
        'EUR' as currency,

        -- Funder struct
        struct(
            CONCAT('https://openalex.org/F', f.funder_id) as id,
            f.display_name,
            f.ror_id,
            f.doi
        ) as funder,

        -- Funding type - map Research.fi types
        CASE
            WHEN LOWER(g.funding_type) LIKE '%fellowship%' THEN 'fellowship'
            WHEN LOWER(g.funding_type) LIKE '%postdoctoral%' THEN 'fellowship'
            WHEN LOWER(g.funding_type) LIKE '%research fellow%' THEN 'fellowship'
            WHEN LOWER(g.funding_type) LIKE '%academy professor%' THEN 'fellowship'
            WHEN LOWER(g.funding_type) LIKE '%centre of excellence%' THEN 'research'
            WHEN LOWER(g.funding_type) LIKE '%flagship%' THEN 'research'
            WHEN LOWER(g.funding_type) LIKE '%infrastructure%' THEN 'infrastructure'
            WHEN LOWER(g.funding_type) LIKE '%strategic%' THEN 'research'
            WHEN LOWER(g.funding_type) LIKE '%academy project%' THEN 'grant'
            ELSE 'grant'
        END as funding_type,

        -- Funder scheme = call programme name
        g.call_programme as funder_scheme,

        -- Provenance
        'researchfi' as provenance,

        -- Dates
        TRY_TO_DATE(g.start_date, 'yyyy-MM-dd') as start_date,
        TRY_TO_DATE(g.end_date, 'yyyy-MM-dd') as end_date,
        TRY_CAST(g.start_year AS INT) as start_year,
        TRY_CAST(g.end_year AS INT) as end_year,

        -- Lead investigator
        CASE
            WHEN g.pi_family_name IS NOT NULL THEN
                struct(
                    g.pi_given_name as given_name,
                    g.pi_family_name as family_name,
                    g.pi_orcid as orcid,
                    CAST(NULL AS DATE) as role_start,
                    struct(
                        g.pi_affiliation_name as name,
                        COALESCE(g.pi_affiliation_country, 'Finland') as country,
                        CAST(NULL AS ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>) as ids
                    ) as affiliation
                )
            ELSE NULL
        END as lead_investigator,

        -- Co-lead and other investigators (not available in Research.fi API)
        CAST(NULL AS STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >) as co_lead_investigator,

        CAST(NULL AS ARRAY<STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >>) as investigators,

        -- Landing page URL
        CONCAT('https://research.fi/en/results/funding/', CAST(g.project_id AS STRING)) as landing_page_url,

        -- No DOI for Academy of Finland grants typically
        CAST(NULL AS STRING) as doi,

        -- Works API URL
        concat('https://api.openalex.org/works?filter=awards.id:G', abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(CAST(g.project_id AS STRING))))) % 9000000000) as works_api_url,

        -- Timestamps
        current_timestamp() as created_date,
        current_timestamp() as updated_date

    FROM openalex.awards.academy_of_finland_raw g
    CROSS JOIN aka_funder f
    WHERE g.project_id IS NOT NULL
)

SELECT * FROM awards_transformed;

In [ ]:
%sql
-- Remove previous data for this source before inserting fresh data
DELETE FROM openalex.awards.openalex_awards_raw
WHERE provenance = 'researchfi' AND priority = 24;

-- Insert into openalex_awards_raw with priority
INSERT INTO openalex.awards.openalex_awards_raw
SELECT
    id,
    display_name,
    description,
    funder_id,
    funder_award_id,
    amount,
    currency,
    funder,
    funding_type,
    funder_scheme,
    provenance,
    start_date,
    end_date,
    start_year,
    end_year,
    lead_investigator,
    co_lead_investigator,
    investigators,
    landing_page_url,
    doi,
    works_api_url,
    created_date,
    updated_date,
    24 as priority
FROM openalex.awards.academy_of_finland_awards;

## Verification Queries

In [ ]:
%sql
-- Check row count (should be ~5.4K)
SELECT COUNT(*) as total_awards FROM openalex.awards.academy_of_finland_awards;

In [ ]:
%sql
-- Sample the data
SELECT 
    id,
    display_name,
    funder_award_id,
    funder_scheme,
    funding_type,
    amount,
    start_year,
    end_year,
    lead_investigator
FROM openalex.awards.academy_of_finland_awards 
LIMIT 10;

In [ ]:
%sql
-- Check funder distribution (should all be Academy of Finland)
SELECT funder.display_name, COUNT(*) as cnt
FROM openalex.awards.academy_of_finland_awards
GROUP BY funder.display_name
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funding_type distribution
SELECT funding_type, COUNT(*) as cnt
FROM openalex.awards.academy_of_finland_awards
GROUP BY funding_type
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funder_scheme distribution (top 20 funding programmes)
SELECT funder_scheme, COUNT(*) as cnt
FROM openalex.awards.academy_of_finland_awards
WHERE funder_scheme IS NOT NULL
GROUP BY funder_scheme
ORDER BY cnt DESC
LIMIT 20;

In [ ]:
%sql
-- Check data completeness
SELECT
    COUNT(*) as total,
    COUNT(display_name) as has_title,
    COUNT(description) as has_abstract,
    COUNT(amount) as has_amount,
    COUNT(start_date) as has_start_date,
    COUNT(end_date) as has_end_date,
    COUNT(lead_investigator) as has_pi,
    SUM(CASE WHEN lead_investigator.orcid IS NOT NULL THEN 1 ELSE 0 END) as has_orcid,
    ROUND(COUNT(lead_investigator) * 100.0 / COUNT(*), 1) as pct_with_pi,
    ROUND(COUNT(description) * 100.0 / COUNT(*), 1) as pct_with_abstract,
    ROUND(SUM(amount), 0) as total_funding_eur
FROM openalex.awards.academy_of_finland_awards;

In [ ]:
%sql
-- Check year distribution
SELECT start_year, COUNT(*) as cnt, ROUND(SUM(amount)/1000000, 1) as funding_millions_eur
FROM openalex.awards.academy_of_finland_awards
WHERE start_year IS NOT NULL
GROUP BY start_year
ORDER BY start_year DESC
LIMIT 20;

In [ ]:
%sql
-- Check lead institutions
SELECT 
    lead_investigator.affiliation.name as institution,
    COUNT(*) as cnt
FROM openalex.awards.academy_of_finland_awards
WHERE lead_investigator.affiliation.name IS NOT NULL
GROUP BY lead_investigator.affiliation.name
ORDER BY cnt DESC
LIMIT 20;